In [34]:
# !pip install langchain sentence-transformers faiss-cpu langchain-community langchain-openai transformers -q

In [35]:
# %pip install --upgrade --quiet langchain langchain-community sentence-transformers faiss-cpu transformers|

In [36]:
# %pip install pandas langchain-text-splitters sentence-transformers numpy #Installation now complete.

In [37]:
# %pip install weaviate-client sentence-transformers 

In [38]:
# %pip install torch

In [39]:
# %pip install accelerate

In [40]:
# %pip install "weaviate-client>=3.26.0,<4.0.0"

In [41]:
# %pip install huggingface_hub[hf_xet]

In [42]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from transformers import pipeline
from langchain_community.document_loaders import CSVLoader, DirectoryLoader, PyPDFLoader
import typing
import time


In [43]:
import weaviate
from langchain_community.vectorstores import Weaviate 

In [44]:
# #loader to lead the data into document structure
# loader = DirectoryLoader(path = 'data',glob = '**/*.csv',loader_cls= CSVLoader,show_progress = True)
# #splitter for chunking the docs
# splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)


In [45]:
class Ingestion:
    def __init__(self,model_name: str = "all-MiniLM-L6-v2",k:int =5): 
                    #  loader:DirectoryLoader = loader, 
                    #  splitter: RecursiveCharacterTextSplitter = splitter):
                
        #Loading the documents:
        self.loader = DirectoryLoader(path = 'data',glob = '**/*.pdf',loader_cls= PyPDFLoader,show_progress = True)
        self.docs = self.loader.load()
        #chunking
        self.splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
        self.chunks = self.splitter.split_documents(self.docs)
        #Embeddings
        self.model_name = model_name
        self.embedding_generator = SentenceTransformerEmbeddings(model_name = self.model_name)
        #Crceating vectorstore 
        self.vectorstore = self.initialize_weaviate()
        # self.initialize_weaviate()
        
        self.retriever = self.vectorstore.as_retriever(search_kwargs = {'k':k})
        


    #initializing weaviate
    def initialize_weaviate(self):
        try:
            # self.client = weaviate.connect_to_local(
            #      host = 'localhost',
            #      port = 8080
            # )

            self.client = weaviate.Client(
                    url="http://localhost:8080",
                    timeout_config=(5, 15)  # (connect timeout, read timeout)
                )

            # we first need to <docker compose up -d> weaviate  
            if not self.client.is_ready():
                    raise ConnectionError("Weaviate is not ready. Ensure Docker container is running: docker compose up -d")

            print("Connected to Weaviate")

            return Weaviate(
                client = self.client,
                index_name = 'Chunk',
                text_key = 'text',
                embedding = self.embedding_generator,
                by_text = False 
            )
        except Exception as e:
             raise ConnectionError(f"Failed to connect to Weaviate: {e}\nMake sure to run: docker compose up -d")
            # print(f"Error: {e}\n Failed to connect to weaviate") 

    #Uploading the generated embeddings into weaviate:-

    def upload_vectors(self, skip_if_exists: bool = True):
        try:
            if skip_if_exists:
                # Check if the Chunk class exists first
                try:
                    schema = self.client.schema.get()
                    class_names = [c['class'] for c in schema.get('classes', [])]
                    
                    if 'Chunk' in class_names:
                        # Class exists, check count
                        result = self.client.query.aggregate("Chunk").with_meta_count().do()
                        existing_count = result.get('data', {}).get('Aggregate', {}).get('Chunk', [{}])[0].get('meta', {}).get('count', 0)
                        
                        if existing_count > 0:
                            print(f"⚠ Weaviate already contains {existing_count} chunks. Skipping upload.")
                            print("To force re-upload, delete the class first or set skip_if_exists=False")
                            return
                    else:
                        print("ℹ 'Chunk' class doesn't exist yet. Creating and uploading...")
                        
                except Exception as e:
                    print(f"ℹ Could not check existing data: {e}. Proceeding with upload...")

            # Upload documents - LangChain will create the schema automatically
            print(f"Uploading {len(self.chunks)} chunks to Weaviate...")
            self.vectorstore.add_documents(self.chunks)
            print(f"✓ Successfully uploaded {len(self.chunks)} chunks to Weaviate")
            
        except Exception as e:
            print(f"✗ Error uploading vectors: {e}")
            raise


    # def upload_vectors(self, skip_if_exists: bool = True):

    #     if skip_if_exists:
    #             result = self.client.query.aggregate("Chunk").with_meta_count().do()
    #             existing_count = result.get('data', {}).get('Aggregate', {}).get('Chunk', [{}])[0].get('meta', {}).get('count', 0)
                
    #             if existing_count > 0:
    #                 print(f"⚠ Weaviate already contains {existing_count} chunks. Skipping upload.")
    #                 print("To force re-upload, delete the class first or set skip_if_exists=False")
    #                 return

    #     self.vectorstore.add_documents(self.chunks)
    #     #Langchain weaviate needs only this one line of code for adding chunks
    #     print(f"Successfully uploaded {len(self.chunks)} into Weaviate")

    def delete_all_vectors(self, class_name = 'Chunk'):
        """Delete all vectors from Weaviate by deleting the Chunk class"""
        try:
            schema = self.client.schema.get()
            class_names = [c['class'] for c in schema.get('classes', [])]
            
            if class_name in class_names:
                self.client.schema.delete_class(class_name)
                print(f"✓ Deleted all vectors ({class_name} class removed)")
            else:
                print(f"ℹ No {class_name} class found. Nothing to delete.")
        except Exception as e:
            print(f"✗ Error deleting vectors: {e}")
    

In [46]:

def ingestion():
    ingest = Ingestion()
    # ingest.initialize_weaviate()
    ingest.delete_all_vectors()
    ingest.upload_vectors()
# ingestion()

In [47]:
class LLM():
    def __init__(self,prompt,llm_name = 'microsoft/Phi-3-mini-4k-instruct'):#meta-llama/Llama-3.2-3B-Instruct# google/gemma-2b
        self.llm_name = llm_name
        self.prompt = prompt
        self.response = None
        try:
            print(f"Loading Model:{self.llm_name}... This may take some time ")
            
            self.llm_model = pipeline(
                'text-generation', 
                model = self.llm_name,
                # device_map = 'auto',
                max_length = 2048,
                )
            
            print("Generating response...")

            result = self.llm_model(
                self.prompt, max_new_tokens = 200 , 
                num_return_sequences = 1, 
                do_sample = True, 
                temperature = 0.6, 
                top_p = 0.9,
                return_full_text=True,
                pad_token_id=50256,
                truncation=True
                )
            
            full_text = result[0]['generated_text']
            # Remove the prompt part
            self.response = full_text[len(self.prompt):].strip()
            print("Response Generated Successfully")
            
        except Exception as e:
            print(f'error: {e}')
            self.response = f"Failed to generate a response.(Error: {e})"
    
    def answer_query(self):
        print(f"\n{'='*50}")
        print(f"ANSWER:\n{self.response}")
        print(f"{'='*50}\n")



In [48]:
class Query(Ingestion):
    def __init__(self):
        # Creating the retriver
        super().__init__()
        # self.retriever = self.vectorstore.as_retriever(search_kwargs = {'k':k})
        self.query = None
        # self.context = self.get_context()
        self.context = None 
        # self.prompt = self.generate_prompt()
        self.prompt = None
    
    def get_query(self):
        try:
            inp = input("Enter any cricket related question(ask it in short): ")
            if inp.strip():
                self.query = inp
            else:
                raise ValueError("Empty Input")
        except Exception as e:
            self.query = "What is the importance of toss in Cricket"
            print(f"Error: {e}\n\n Therefore, using default query")

    def get_context(self):
        retrieved_docs = self.retriever.invoke(self.query)# newer version sue get_relevant_documents
        context = "\n\n".join([d.page_content for d in retrieved_docs])
        self.context = context
    
    def generate_prompt(self):
        self.prompt =  f"""You are a cricket expert. Answer the question on the basis of the context below.
        Respond in point format if multiple lines are there. Format the answer properly.
        Context:{self.context}
        Question:{self.query}

        Answer:"""
    
    def generate_response(self):
        llm = LLM(self.prompt)
        llm.answer_query()
    

In [49]:
def ask_rag(query: Query):
    query.get_query()   #It takes the user Query
    query.get_context() #It generates the context for the Specific query
    query.generate_prompt()
    query.generate_response()
    

In [50]:
query = Query()

100%|██████████| 2/2 [00:19<00:00,  9.87s/it]


Connected to Weaviate


In [52]:
ask_rag(query)

Loading Model:microsoft/Phi-3-mini-4k-instruct... This may take some time 


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.21it/s]
Device set to use cpu


Generating response...
Response Generated Successfully

ANSWER:
- A ball is called a no ball if it is delivered in breach of clause 20.4.2.7 (41.4 -	Deliberate attempt to distract striker before playing the ball or 41.5 -	Deliberate attempt of Distraction, Deception or Obstruction of striker after the stroke).
        - A ball is called a no ball to over-ride a wide ball.
        - If a fielder's thrown ball makes contact with a camera on or over the field of play, the umpire shall call and signal a dead ball.
        - If a no ball is called following a check by the third umpire, the batting side shall benefit from the reversal of the dismissal and the one run for the No ball, but shall not benefit from any runs that may have accrued from the delivery had the on-field umpire originally

